# Load Packages

In [ ]:

%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
import torch
sys.path.append("../../")

from src.file_manager.load_data import load_split_dict
from src.models.bnn.tuning import tune_bnn_model
from src.models.bnn.training import train_model_w_best_param
from src.models.bnn.predicting import bnn_model_prediction
from src.misc import create_folder
# from seed_file import seed

seed = 2023
tuning_seed = 2023
data_label = "physionet"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_data_folder = join(fp_project_folder, "../", "data")
fp_output_data_folder = join(fp_data_folder, data_label)
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")

# Seed filepaths
fp_cur_tune_folder = join(fp_tuning, str(tuning_seed))
create_folder(fp_cur_tune_folder)
fp_cur_model_folder = join(fp_models, str(seed))
create_folder(fp_cur_model_folder)
fp_cur_predictions_folder = join(fp_predictions, str(seed))
create_folder(fp_cur_predictions_folder)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [6]:
split_dict = load_split_dict(fp_output_data_folder)

# Tune BNN

In [13]:
if seed==tuning_seed:
    all_bnn_best_hp = {}
    for time_label, target_cols in tqdm(split_dict["target_cols"].items()):
        fp_model = join(fp_cur_model_folder, f"bnn_{time_label}.pt")
        bnn_tuning_df, bnn_best_hp = tune_bnn_model(
            param_grid={"num_layers":[2, 3], "width":[32, 64, 128, 256]}, 
            train_df=split_dict["train_df"], valid_df=split_dict["valid_df"], 
            feat_cols=split_dict["feat_cols"], target_cols=target_cols, 
            epochs=500, patience=5, seed=seed, batch_size=batch_size, fp_model=fp_model)
        display(bnn_tuning_df)
        bnn_tuning_df.to_csv(join(fp_cur_tune_folder, f"tuning_bnn_{time_label}.csv"))
        all_bnn_best_hp[time_label] = bnn_best_hp
    joblib.dump(all_bnn_best_hp, join(fp_tuning, "all_bnn_best_hp.joblib"))
    display(all_bnn_best_hp)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

Early stopping! Model achieved best performance at Epoch 82 with loss = 0.00666826979117015.


  0%|          | 0/500 [00:00<?, ?it/s]

...

Early stopping! Model achieved best performance at Epoch 55 with loss = 0.01068337261428806.


  0%|          | 0/500 [00:00<?, ?it/s]

Early stopping! Model achieved best performance at Epoch 0 with loss = 0.029261919465215457.


,num_layers,width,loss,epoch,time/s,best_hyperparameter
0,2,32,0.006777,76,512.340670,False
1,2,64,0.006770,90,592.558297,False
2,2,128,0.008211,157,1004.926380,False
3,2,256,0.020846,37,263.166778,False
4,3,32,0.006024,92,594.800603,True
5,3,64,0.006235,103,675.051737,False
6,3,128,0.010683,55,380.705074,False
7,3,256,0.029262,0,44.068197,False


{'t+1': {'num_layers': 3, 'width': 32},
 't+2': {'num_layers': 3, 'width': 32},
 't+3': {'num_layers': 3, 'width': 32}}

# Training

In [ ]:
all_bnn_best_hp = joblib.load(join(fp_tuning, "all_bnn_best_hp.joblib"))
for time_label, time_info_dict in tqdm(split_dict["target_cols"].items()):
    fp_model = join(fp_cur_model_folder, f"bnn_{time_label}.pt")
    bnn_best_hp = all_bnn_best_hp[time_label]
    bnn_model = train_model_w_best_param(
        best_param=bnn_best_hp, 
        train_df=split_dict["train_df"], valid_df=split_dict["valid_df"], 
        feat_cols=split_dict["feat_cols"], target_cols=target_cols,
        epochs=500, patience=5, seed=seed, fp_model=fp_model
    )

# Prediction

In [ ]:
for time_label, time_info_dict in tqdm(split_dict["target_cols"].items()):
    fp_model = join(fp_cur_model_folder, f"bnn_{time_label}.pt")
    bnn_model = torch.load(fp_model)
    bnn_valid_df = bnn_model_prediction(
        bnn_model, split_dict["valid_df"], 
        feat_cols=split_dict["feat_cols"], target_cols=target_cols, 
        T=10, seed=seed, regressor_label=time_label)
    bnn_test_df = bnn_model_prediction(
        bnn_model, split_dict["test_df"], 
        feat_cols=split_dict["feat_cols"], target_cols=target_cols, 
        T=10, seed=seed, regressor_label=time_label)
    # display(bnn_test_df)
    bnn_valid_df.to_csv(join(fp_cur_predictions_folder, f"bnn_valid_{time_label[-1]}.csv"))
    bnn_test_df.to_csv(join(fp_cur_predictions_folder, f"bnn_test_{time_label[-1]}.csv"))